In [69]:
import hashlib
import hmac
import json
import time
import requests
import os
import pandas as pd
import numpy as np
from datetime import datetime

def gen_sign(api_secret, payload_string=None):
    return hmac.new(api_secret.encode('utf-8'), payload_string.encode('utf-8'), hashlib.sha256).hexdigest()

def gen_query_param(url, query_param):
    req = requests.PreparedRequest()
    req.prepare_url(url, query_param)
    return req.url.replace(url,"")

def gen_ts():

    host = 'https://api.bitkub.com'
    path = '/api/v3/servertime'

    ts = str(round(time.time() * 1000))
    param = {
    }
    query_param = gen_query_param(host+path, param)

    payload = []
    payload.append(ts)
    payload.append('GET')
    payload.append(path)
    payload.append(query_param)

    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'X-BTK-TIMESTAMP': ts,
    }

    response_ts = requests.request('GET', f'{host}{path}{query_param}', headers=headers, data={}, verify=True)
    return response_ts

In [91]:
# call api for my-order-history

def call_my_order_history_each_coin(sym):
    
    host = 'https://api.bitkub.com'
    path = '/api/v3/market/my-order-history'
    api_key = os.environ['BITKUB_API_KEY']
    api_secret = os.environ['BITKUB_API_SECRET']

    # ts = str(round(time.time() * 1000))
    ts = str(gen_ts().json())

    param = {
        'sym':sym, # symbol in quote_base format: e.g. btc_thb
        'start': str(round(int(datetime.strptime("13-11-2024", "%d-%m-%Y").timestamp()) * 1000)),
        'lmt': 1000
    }

    query_param = gen_query_param(host+path, param)

    payload = []
    payload.append(ts)
    payload.append('GET')
    payload.append(path)
    payload.append(query_param)

    sig = gen_sign(api_secret, ''.join(payload))
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'X-BTK-TIMESTAMP': ts,
        'X-BTK-SIGN': sig,
        'X-BTK-APIKEY': api_key
    }

    response_my_order_history = requests.request('GET', f'{host}{path}{query_param}', headers=headers, data={}, verify=True)
    return response_my_order_history

In [92]:
# call api for all symbols

def call_all_symbols():

    host = 'https://api.bitkub.com'
    path = '/api/market/symbols'

    ts = str(round(time.time() * 1000))
    param = {
    }
    query_param = gen_query_param(host+path, param)

    payload = []
    payload.append(ts)
    payload.append('GET')
    payload.append(path)
    payload.append(query_param)

    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'X-BTK-TIMESTAMP': ts,
    }

    response_all_symbols = requests.request('GET', f'{host}{path}{query_param}', headers=headers, data={}, verify=True)
    return response_all_symbols

In [93]:
# convert to df for all symbols
data = call_all_symbols().json()
result = data['result']
df_all_symbols = pd.DataFrame(result)
df_all_symbols['symbol'] = df_all_symbols['symbol'].str.lower().str.split('_').str[::-1].str.join('_')
df_all_symbols['symbol']

0       btc_thb
1       eth_thb
2       ada_thb
3       bch_thb
4      usdt_thb
         ...   
156    tnsr_thb
157     dbr_thb
158    kaia_thb
159    carv_thb
160     fet_thb
Name: symbol, Length: 161, dtype: object

In [94]:
# convert to df for my-order-history

df_my_order_history_all = pd.DataFrame()

for symbol in df_all_symbols['symbol']:
    
    sym = symbol
    data = call_my_order_history_each_coin(sym).json()
    result = data['result']

    if result == [] or result == None:
        continue

    print(sym)
    df_my_order_history = pd.DataFrame(result)
    df_my_order_history['ts'] = pd.to_datetime(df_my_order_history['ts'], unit='ms')
    df_my_order_history_all = pd.concat([df_my_order_history_all, df_my_order_history], ignore_index=True)

df_my_order_history_all.head()

btc_thb
ada_thb
bch_thb
xrp_thb
ctxc_thb
xlm_thb
six_thb
doge_thb
ksm_thb
velo_thb


,txn_id,order_id,hash,parent_order_id,parent_order_hash,super_order_id,super_order_hash,client_id,taken_by_me,is_maker,side,type,rate,fee,credit,amount,ts
0,BTCSELL0024178421,298473336,E5a9FBF5rPEWA7U1VncukDmqSmPumu,0,JiWPghZbgX9MjqZ9H2K,0,JiWPghZbgX9MjqZ9H2K,,False,False,sell,limit,3267709.04,22.15,0,0.00271022,2024-11-26 05:22:37.898
1,BTCBUY0024104677,357906601,3xs6wsMQkpBK1vtRR261Lv2KtdomS,0,51o5gooMYcNqCWwA32,0,51o5gooMYcNqCWwA32,,False,False,buy,limit,3388999.95,23.02,0,9207.98,2024-11-22 01:54:05.409
2,BTCBUY0024060282,297277680,E5a9FBF5rPEWA7SuX6GcRNpieStUTu,0,JiWPghZbgX9MjqZ9H2K,0,JiWPghZbgX9MjqZ9H2K,,False,True,sell,limit,3210000,46.69,0,0.00581791,2024-11-20 07:40:20.438
3,BTCSELL0024059027,357335554,3xs6wsMQkpBK1vtR5WKpQU5vSHuQc,0,51o5gooMYcNqCWwA32,0,51o5gooMYcNqCWwA32,,False,True,buy,limit,3186000,46.46,0,18582.35,2024-11-20 05:02:45.788
4,BTCBUY0024034030,296826830,E5a9FBF5rPEWA7RqXB4gowQnQLXs2B,296825870,E5a9FBF5rPEWA7RqXB3wza19EdZs9M,296825807,E5a9FBF5rPEWA7RqXB3wzJgGKVKtmR,,False,True,sell,limit,3162000,1.08,0,0.00013607,2024-11-18 11:34:28.226


In [95]:
# transform data amount_thb
df_my_order_history_all['rate'] = pd.to_numeric(df_my_order_history_all['rate'])
df_my_order_history_all['amount'] = pd.to_numeric(df_my_order_history_all['amount'])
df_my_order_history_all['amount_thb'] = np.where(df_my_order_history_all['side'] == 'sell', df_my_order_history_all['amount'] * df_my_order_history_all['rate'], df_my_order_history_all['amount'])
df_my_order_history_all['amount_thb'] = np.round(df_my_order_history_all['amount_thb'], 2)

# transform data ticker
df_my_order_history_all['ticker'] = df_my_order_history_all['txn_id'].str.split('(BUY|SELL)').str[0]
df_my_order_history_all.head()

,txn_id,order_id,hash,parent_order_id,parent_order_hash,super_order_id,super_order_hash,client_id,taken_by_me,is_maker,side,type,rate,fee,credit,amount,ts,amount_thb,ticker
0,BTCSELL0024178421,298473336,E5a9FBF5rPEWA7U1VncukDmqSmPumu,0,JiWPghZbgX9MjqZ9H2K,0,JiWPghZbgX9MjqZ9H2K,,False,False,sell,limit,3267709.04,22.15,0,0.002710,2024-11-26 05:22:37.898,8856.21,BTC
1,BTCBUY0024104677,357906601,3xs6wsMQkpBK1vtRR261Lv2KtdomS,0,51o5gooMYcNqCWwA32,0,51o5gooMYcNqCWwA32,,False,False,buy,limit,3388999.95,23.02,0,9207.980000,2024-11-22 01:54:05.409,9207.98,BTC
2,BTCBUY0024060282,297277680,E5a9FBF5rPEWA7SuX6GcRNpieStUTu,0,JiWPghZbgX9MjqZ9H2K,0,JiWPghZbgX9MjqZ9H2K,,False,True,sell,limit,3210000.00,46.69,0,0.005818,2024-11-20 07:40:20.438,18675.49,BTC
3,BTCSELL0024059027,357335554,3xs6wsMQkpBK1vtR5WKpQU5vSHuQc,0,51o5gooMYcNqCWwA32,0,51o5gooMYcNqCWwA32,,False,True,buy,limit,3186000.00,46.46,0,18582.350000,2024-11-20 05:02:45.788,18582.35,BTC
4,BTCBUY0024034030,296826830,E5a9FBF5rPEWA7RqXB4gowQnQLXs2B,296825870,E5a9FBF5rPEWA7RqXB3wza19EdZs9M,296825807,E5a9FBF5rPEWA7RqXB3wzJgGKVKtmR,,False,True,sell,limit,3162000.00,1.08,0,0.000136,2024-11-18 11:34:28.226,430.25,BTC


In [115]:
df_my_order_history_all = df_my_order_history_all.sort_values(by=['ts'], ascending=False)
df_my_order_history_all[df_my_order_history_all['ticker'] == 'VELO']

,txn_id,order_id,hash,parent_order_id,parent_order_hash,super_order_id,super_order_hash,client_id,taken_by_me,is_maker,side,type,rate,fee,credit,amount,ts,amount_thb,ticker
58,VELOBUY0000614273,3973350,fwQ6dnXsCjJ1zRX1bhNduBNqjUw,3973327,fwQ6dnXsCjJ1zRX1bhG6wk9uRib,3973327,fwQ6dnXsCjJ1zRX1bhG6wk9uRib,,False,True,sell,limit,4.30,11.19,0,1040.021265,2024-11-26 17:03:28.456,4472.09,VELO
59,VELOBUY0000614271,3973327,fwQ6dnXsCjJ1zRX1bhG6wk9uRib,0,JiWPghcwrqb9rV7drCj,0,JiWPghcwrqb9rV7drCj,,False,True,sell,limit,4.30,34.53,0,3211.176744,2024-11-26 17:03:03.448,13808.06,VELO
60,VELOSELL0000613940,4987483,9pfmKkr8kSScN3qLVjTUy7i4wi,4987482,9pfmKkr8kSScN3qLVjTUraBEoS,4987337,9pfmKkr8kSScN3qLVhDX8wd1ip,,False,True,buy,limit,4.15,6.8,0,2719.860000,2024-11-26 14:57:44.558,2719.86,VELO
61,VELOSELL0000613939,4987482,9pfmKkr8kSScN3qLVjTUraBEoS,4987337,9pfmKkr8kSScN3qLVhDX8wd1ip,4987337,9pfmKkr8kSScN3qLVhDX8wd1ip,,False,True,buy,limit,4.15,17.01,0,6819.020000,2024-11-26 14:57:44.195,6819.02,VELO
62,VELOSELL0000613938,4987337,9pfmKkr8kSScN3qLVhDX8wd1ip,0,51o5gop7YEGtrGnfAY,0,51o5gop7YEGtrGnfAY,,False,True,buy,limit,4.15,5.49,0,2200.030000,2024-11-26 14:57:43.380,2200.03,VELO
63,VELOBUY0000613881,4987168,9pfmKkr8kSScN3qLVcr4A3jV9A,0,51o5gop7YEGtrGnfAY,0,51o5gop7YEGtrGnfAY,,False,False,buy,limit,4.11,2.35,0,941.120000,2024-11-26 14:47:15.737,941.12,VELO
64,VELOSELL0000613847,4987022,9pfmKkr8kSScN3qLVaca5cQd28,0,51o5gop7YEGtrGnfAY,0,51o5gop7YEGtrGnfAY,,False,True,buy,limit,4.07,12.23,0,4901.120000,2024-11-26 14:40:31.594,4901.12,VELO


In [123]:
# create df for win and loss trade analysis

amount_thb_accum = df_my_order_history_all.groupby('super_order_id')['amount_thb'].sum().reset_index()
df_my_order_history_anal = df_my_order_history_all.merge(amount_thb_accum, how='left', left_on='order_id', right_on='super_order_id', suffixes=('', '_child'))
df_my_order_history_anal['amount_thb_child'] = df_my_order_history_anal['amount_thb_child'].fillna(0)
df_my_order_history_anal['amount_thb_total'] = df_my_order_history_anal['amount_thb'] + df_my_order_history_anal['amount_thb_child']
df_my_order_history_anal = df_my_order_history_anal[df_my_order_history_anal['super_order_id'] == '0']
df_my_order_history_anal = df_my_order_history_anal[['order_id', 'ticker', 'amount_thb_total', 'side', 'type', 'rate', 'ts']]
df_my_order_history_anal.sort_values(by=['ticker', 'ts'], ascending=True)

,order_id,ticker,amount_thb_total,side,type,rate,ts
25,145959168,ADA,9782.22,buy,limit,3.392000e+01,2024-11-26 06:20:19.568
22,125809743,ADA,8771.05,sell,market,3.049000e+01,2024-11-26 11:34:13.382
43,136370725,BCH,9207.97,buy,limit,1.683400e+04,2024-11-22 01:47:08.595
37,109805047,BCH,9930.28,sell,limit,1.820000e+04,2024-11-23 04:47:09.667
13,137040126,BCH,9117.21,buy,market,1.719716e+04,2024-11-27 03:10:36.825
8,110358126,BCH,9087.45,sell,market,1.718400e+04,2024-11-27 09:31:58.975
64,355307688,BTC,18093.00,buy,limit,3.035000e+06,2024-11-13 09:19:54.062
63,295839223,BTC,18731.60,sell,limit,3.150000e+06,2024-11-14 04:54:26.099
62,355597836,BTC,18684.77,buy,limit,3.160000e+06,2024-11-14 07:49:55.353
61,295942573,BTC,18755.99,sell,limit,3.180000e+06,2024-11-14 13:58:35.681
